In [1]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext, Window
from pyspark.sql.functions import *
from pyspark.sql.types import ArrayType, IntegerType

In [2]:
# Config setups
conf = SparkConf()  # create the configuration

bucket = "hackathon.jjkoh.com"
conf.set('temporaryGcsBucket', bucket)
conf.set('spark.app.name', 'spark-grab-challenge')

sc = SparkContext.getOrCreate()
spark = SparkSession(sc).builder.config(conf=conf).getOrCreate()

In [3]:
def print_info(df, line_to_show=5):
    if df is not None:
        if isinstance(df, dict):
            for key in df:
                print(key)
                df[key].printSchema()
                if line_to_show > 0:
                    df[key].show(line_to_show)
        else:
            df.printSchema()
            if line_to_show > 0:
                df.show(line_to_show)
    else:
        pass

In [4]:
# Load data from BigQuery.
df = spark.read.format('bigquery') \
  .option('table', 'jjkoh95:jjkoh.grab_raw_singapore') \
  .load()

In [5]:
# Filter 78371, 77844, 65738, 72549 (Outlier)
df = df.filter(~df['trj_id'].isin([78371, 77844, 65738, 72549]))

In [6]:
# Sort data and filter
df = df.sort('trj_id', 'pingtimestamp')

df = df.withColumn("idx", monotonically_increasing_id())
windowSpec = Window.partitionBy('trj_id').orderBy("idx")
df = df.withColumn("idx", row_number().over(windowSpec))
df = df.withColumn("remainder", col('idx') % 60)
df = df.filter(df.remainder == 0)

In [7]:
# print_info(df.limit(10))

In [8]:
# Sort data and filter
df = df.sort('trj_id', 'pingtimestamp')

# New column with count number
windowSpec = Window.partitionBy('trj_id')
duplicate_df = df.withColumn('explode', count('trj_id').over(windowSpec))

# Make the count number as an array with the number of count number
n_to_array = udf(lambda n : [n] * n, ArrayType(IntegerType()))
duplicate_df = duplicate_df.withColumn('explode', n_to_array(duplicate_df.explode))

# Explore array and delete the explode column
duplicate_df = (duplicate_df
                .withColumn('explode', explode(duplicate_df.explode))
                .drop('explode')
               )

# Add index
duplicate_df = duplicate_df.withColumn("idx", monotonically_increasing_id())
windowSpec = Window.partitionBy('trj_id').orderBy("idx")
duplicate_df = duplicate_df.withColumn("idx", row_number().over(windowSpec))

# print_info(duplicate_df)

In [12]:
# duplicate_df.count()

In [9]:
# Reorder table
duplicate_df2 = duplicate_df.withColumn('idx', monotonically_increasing_id())
windowSpec = Window.partitionBy('trj_id', 'pingtimestamp').orderBy('idx')
duplicate_df2 = (duplicate_df2
                 .withColumn('idx', row_number().over(windowSpec))
                 .sort('trj_id', 'idx')
                 .drop('idx')
                )

# Rename column
for name in duplicate_df2.schema.names:
    duplicate_df2 = duplicate_df2.withColumnRenamed(name, name + "2")

# Add index
duplicate_df2 = duplicate_df2.withColumn("idx2", monotonically_increasing_id())
windowSpec = Window.partitionBy('trj_id2').orderBy("idx2")
duplicate_df2 = duplicate_df2.withColumn("idx2", row_number().over(windowSpec))

# print_info(duplicate_df2)

In [10]:
# duplicate_df2.count()

In [11]:
final_df = duplicate_df.join(duplicate_df2, (duplicate_df.idx == duplicate_df2.idx2) & (duplicate_df.trj_id == duplicate_df2.trj_id2))
final_df = (final_df
            .filter(col("pingtimestamp") < col("pingtimestamp2"))
            .select('rawlat', 'rawlng', 'pingtimestamp', 'rawlat2', 'rawlng2', 'pingtimestamp2')
           )

# print_info(final_df)

In [12]:
# final_df.count()

In [13]:
# # Saving the data to BigQuery
# final_df.write.format('bigquery') \
#     .option('table', 'jjkoh95:jjkoh.transform_grab_singapore_3_trips') \
#     .save()

In [ ]:
OUTPUT_PATH = 'gs://hackathon.jjkoh.com/pipeline-dataproc-3'

output_directory = OUTPUT_PATH
output_files = output_directory + '/part-*'

sql_context = SQLContext(sc)
(final_df.write.option('header','true').format('csv').save(output_directory))

In [19]:
!gsutil ls gs://hackathon.jjkoh.com

gs://hackathon.jjkoh.com/traffic-cluster-density-20-kmeans.pkl
gs://hackathon.jjkoh.com/traffic-cluster-speed-20-kmeans.pkl
gs://hackathon.jjkoh.com/dataflow/
gs://hackathon.jjkoh.com/pipeline-dataproc-2/
gs://hackathon.jjkoh.com/pipeline-dataproc-3/
gs://hackathon.jjkoh.com/pipeline-dataproc/
gs://hackathon.jjkoh.com/pipeline/
gs://hackathon.jjkoh.com/pipelinetrip/
gs://hackathon.jjkoh.com/pipelinetrip1/
gs://hackathon.jjkoh.com/raw-csv-jakarta/
gs://hackathon.jjkoh.com/raw-csv-singapore/
gs://hackathon.jjkoh.com/raw-parquet-jakarta/
gs://hackathon.jjkoh.com/raw-parquet-singapore/
gs://hackathon.jjkoh.com/tmp/


In [21]:
!gsutil ls gs://hackathon.jjkoh.com/pipeline-dataproc-3/

gs://hackathon.jjkoh.com/pipeline-dataproc-3/
gs://hackathon.jjkoh.com/pipeline-dataproc-3/_SUCCESS
gs://hackathon.jjkoh.com/pipeline-dataproc-3/part-00000-d4826f74-e12f-44ea-96bb-7e21e5a02d40-c000.csv
